In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
import os
import shutil

In [7]:
input_path = r'C:\Users\Wojtek\Documents\Projects\dog-expert\datasets\train'
csv_path = r'C:\Users\Wojtek\Documents\Projects\dog-expert\datasets\labels.csv'
output_path = r'C:\Users\Wojtek\Documents\Projects\dog-expert\datasets\stanford'

In [8]:
def get_stem(filename):
    stem = Path(filename).stem
    return stem

In [9]:
def organize_classes(input_path, output_path, file_dict):
    breeds_found = set()
    for file_stem, breed in file_dict.items():
        if breed not in breeds_found:
            if not os.path.exists(os.path.join(output_path, breed)):
                os.makedirs(os.path.join(output_path, breed))
            breeds_found.add(breed)
        shutil.copy( os.path.join( input_path, ''.join( [file_stem, '.jpg'] ) ), os.path.join( output_path, breed, ''.join([file_stem, '.jpg']) ))
    return None


In [10]:
labels = pd.read_csv(csv_path)

In [11]:
file_dict = dict(zip(labels['id'].values, labels['breed'].values))

In [12]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [13]:
a = organize_classes(input_path, output_path, file_dict)

In [9]:
train_data

NameError: name 'train_data' is not defined